https://www.kaggle.com/code/lorenzodifolco/premier-league-2000-2018-analysis

Use SQL to find pts, gd, gs, gc, wins, losses, draws 

In [76]:
import pandas as pd 
import xgboost as xgb 
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC 
import matplotlib.pyplot as plt 


In [77]:
df = pd.read_csv('E0 (1).csv')

In [78]:

df.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,05/08/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,10,10,2,2,16,11,3,5,1,2,0,0,4.20,3.6,1.85,4.33,3.50,1.87,4.30,3.55,1.85,4.50,3.65,1.89,4.40,3.5,1.83,4.60,3.5,1.87,4.60,3.78,1.95,4.39,3.59,1.88,2.10,1.72,2.14,1.78,2.19,1.91,2.09,1.76,0.50,2.04,1.89,2.03,1.89,2.06,1.91,2.01,1.87,4.50,3.60,1.80,4.50,3.5,1.83,4.40,3.55,1.85,4.58,3.63,1.88,4.80,3.4,1.78,4.75,3.5,1.85,5.01,3.70,1.91,4.56,3.57,1.85,2.10,1.72,2.14,1.78,2.19,1.91,2.08,1.76,0.50,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85
1,E0,06/08/2022,12:30,Fulham,Liverpool,2,2,D,1,0,H,A Madley,9,11,3,4,7,9,4,4,2,0,0,0,11.00,6.0,1.25,10.00,5.75,1.28,12.00,5.75,1.27,11.20,6.22,1.28,12.00,5.5,1.27,13.00,6.0,1.25,13.00,6.40,1.31,10.99,6.05,1.28,1.50,2.62,1.50,2.70,1.54,2.76,1.48,2.63,1.75,1.90,2.03,1.91,2.00,1.92,2.04,1.89,1.99,11.00,5.75,1.28,9.25,6.0,1.29,11.00,5.50,1.30,10.50,6.50,1.29,11.00,5.5,1.27,11.50,6.0,1.29,11.95,6.93,1.30,10.33,6.20,1.28,1.50,2.62,1.49,2.77,1.51,3.00,1.47,2.73,1.75,1.90,2.03,1.91,2.02,2.01,2.06,1.89,1.99
2,E0,06/08/2022,15:00,Bournemouth,Aston Villa,2,0,H,1,0,H,P Bankes,7,15,3,2,18,16,5,5,3,3,0,0,3.75,3.5,2.00,3.75,3.40,2.00,3.65,3.45,2.05,3.93,3.58,2.04,3.75,3.3,2.05,3.75,3.3,2.00,4.00,3.66,2.10,3.80,3.50,2.04,2.00,1.80,2.10,1.81,2.10,1.87,2.03,1.80,0.50,1.87,2.06,1.88,2.04,1.88,2.07,1.85,2.04,4.00,3.50,1.95,3.90,3.4,1.95,3.85,3.45,2.00,4.09,3.59,2.00,4.00,3.4,1.95,4.10,3.4,2.00,4.25,3.63,2.06,3.99,3.49,2.00,2.10,1.72,2.13,1.79,2.24,1.81,2.10,1.76,0.50,1.93,2.00,1.93,2.00,1.94,2.04,1.88,2.00
3,E0,06/08/2022,15:00,Leeds,Wolves,2,1,H,1,1,D,R Jones,12,15,4,6,13,9,6,4,2,0,0,0,2.25,3.4,3.20,2.30,3.30,2.95,2.30,3.30,3.15,2.39,3.33,3.30,2.25,3.3,3.20,2.30,3.2,3.10,2.42,3.54,3.30,2.34,3.34,3.18,2.05,1.85,2.09,1.83,2.11,1.87,2.03,1.81,-0.25,2.05,1.88,2.04,1.89,2.06,1.90,2.01,1.87,2.37,3.30,3.00,2.40,3.3,2.75,2.45,3.30,2.95,2.45,3.44,3.09,2.40,3.3,2.90,2.40,3.4,3.00,2.50,3.55,3.18,2.43,3.36,3.02,1.95,1.95,1.96,1.94,2.09,1.96,1.96,1.87,-0.25,2.08,1.85,2.10,1.84,2.14,1.87,2.08,1.81
4,E0,06/08/2022,15:00,Newcastle,Nott'm Forest,2,0,H,0,0,D,S Hooper,23,5,10,0,9,14,11,1,0,3,0,0,1.66,3.8,5.25,1.65,3.80,5.50,1.65,3.80,5.50,1.71,3.74,5.83,1.67,3.7,5.25,1.62,3.7,5.50,1.72,3.96,6.00,1.67,3.80,5.57,2.05,1.85,2.10,1.81,2.10,1.86,2.03,1.81,-0.75,1.87,2.06,1.92,2.01,1.92,2.08,1.86,2.02,1.53,4.00,6.00,1.58,3.9,6.00,1.63,3.80,6.00,1.57,4.22,6.60,1.53,3.9,6.50,1.57,3.9,7.00,1.67,4.30,7.00,1.59,4.07,6.15,1.94,1.96,1.97,1.93,2.06,1.97,1.94,1.89,-1.00,1.97,1.96,1.99,1.93,2.19,1.97,2.03,1.86


In [79]:
df.columns

Index(['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG',
       ...
       'AvgC<2.5', 'AHCh', 'B365CAHH', 'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH',
       'MaxCAHA', 'AvgCAHH', 'AvgCAHA'],
      dtype='object', length=106)

In [80]:
df.iloc[:5,:]

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,05/08/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,10,10,2,2,16,11,3,5,1,2,0,0,4.20,3.6,1.85,4.33,3.50,1.87,4.30,3.55,1.85,4.50,3.65,1.89,4.40,3.5,1.83,4.60,3.5,1.87,4.60,3.78,1.95,4.39,3.59,1.88,2.10,1.72,2.14,1.78,2.19,1.91,2.09,1.76,0.50,2.04,1.89,2.03,1.89,2.06,1.91,2.01,1.87,4.50,3.60,1.80,4.50,3.5,1.83,4.40,3.55,1.85,4.58,3.63,1.88,4.80,3.4,1.78,4.75,3.5,1.85,5.01,3.70,1.91,4.56,3.57,1.85,2.10,1.72,2.14,1.78,2.19,1.91,2.08,1.76,0.50,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85
1,E0,06/08/2022,12:30,Fulham,Liverpool,2,2,D,1,0,H,A Madley,9,11,3,4,7,9,4,4,2,0,0,0,11.00,6.0,1.25,10.00,5.75,1.28,12.00,5.75,1.27,11.20,6.22,1.28,12.00,5.5,1.27,13.00,6.0,1.25,13.00,6.40,1.31,10.99,6.05,1.28,1.50,2.62,1.50,2.70,1.54,2.76,1.48,2.63,1.75,1.90,2.03,1.91,2.00,1.92,2.04,1.89,1.99,11.00,5.75,1.28,9.25,6.0,1.29,11.00,5.50,1.30,10.50,6.50,1.29,11.00,5.5,1.27,11.50,6.0,1.29,11.95,6.93,1.30,10.33,6.20,1.28,1.50,2.62,1.49,2.77,1.51,3.00,1.47,2.73,1.75,1.90,2.03,1.91,2.02,2.01,2.06,1.89,1.99
2,E0,06/08/2022,15:00,Bournemouth,Aston Villa,2,0,H,1,0,H,P Bankes,7,15,3,2,18,16,5,5,3,3,0,0,3.75,3.5,2.00,3.75,3.40,2.00,3.65,3.45,2.05,3.93,3.58,2.04,3.75,3.3,2.05,3.75,3.3,2.00,4.00,3.66,2.10,3.80,3.50,2.04,2.00,1.80,2.10,1.81,2.10,1.87,2.03,1.80,0.50,1.87,2.06,1.88,2.04,1.88,2.07,1.85,2.04,4.00,3.50,1.95,3.90,3.4,1.95,3.85,3.45,2.00,4.09,3.59,2.00,4.00,3.4,1.95,4.10,3.4,2.00,4.25,3.63,2.06,3.99,3.49,2.00,2.10,1.72,2.13,1.79,2.24,1.81,2.10,1.76,0.50,1.93,2.00,1.93,2.00,1.94,2.04,1.88,2.00
3,E0,06/08/2022,15:00,Leeds,Wolves,2,1,H,1,1,D,R Jones,12,15,4,6,13,9,6,4,2,0,0,0,2.25,3.4,3.20,2.30,3.30,2.95,2.30,3.30,3.15,2.39,3.33,3.30,2.25,3.3,3.20,2.30,3.2,3.10,2.42,3.54,3.30,2.34,3.34,3.18,2.05,1.85,2.09,1.83,2.11,1.87,2.03,1.81,-0.25,2.05,1.88,2.04,1.89,2.06,1.90,2.01,1.87,2.37,3.30,3.00,2.40,3.3,2.75,2.45,3.30,2.95,2.45,3.44,3.09,2.40,3.3,2.90,2.40,3.4,3.00,2.50,3.55,3.18,2.43,3.36,3.02,1.95,1.95,1.96,1.94,2.09,1.96,1.96,1.87,-0.25,2.08,1.85,2.10,1.84,2.14,1.87,2.08,1.81
4,E0,06/08/2022,15:00,Newcastle,Nott'm Forest,2,0,H,0,0,D,S Hooper,23,5,10,0,9,14,11,1,0,3,0,0,1.66,3.8,5.25,1.65,3.80,5.50,1.65,3.80,5.50,1.71,3.74,5.83,1.67,3.7,5.25,1.62,3.7,5.50,1.72,3.96,6.00,1.67,3.80,5.57,2.05,1.85,2.10,1.81,2.10,1.86,2.03,1.81,-0.75,1.87,2.06,1.92,2.01,1.92,2.08,1.86,2.02,1.53,4.00,6.00,1.58,3.9,6.00,1.63,3.80,6.00,1.57,4.22,6.60,1.53,3.9,6.50,1.57,3.9,7.00,1.67,4.30,7.00,1.59,4.07,6.15,1.94,1.96,1.97,1.93,2.06,1.97,1.94,1.89,-1.00,1.97,1.96,1.99,1.93,2.19,1.97,2.03,1.86


In [81]:
df = df.iloc[:,:24]

In [82]:
# Description of Dataset

# Key to results data:

# Div = League Division
# Date = Match Date (dd/mm/yy)
# Time = Time of match kick-off
# HomeTeam = Home Team
# Away team = Away Team
# FTHG and HG = Full Time Home Team Goals
# FTAG and AG = Full-Time Away Team Goals
# FTR and Res = Full-Time Result (H=Home Win, D=Draw, A=Away Win)
# HTHG = Half Time Home Team Goals
# HTAG = Half Time Away Team Goals
# HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)
# Match Statistics (where available)

# Attendance = Crowd Attendance
# Referee = Match Referee
# HS = Home Team Shots
# AS = Away Team Shots
# HST = Home Team Shots on Target
# AST = Away Team Shots on Target
# HHW = Home Team Hit Woodwork
# AHW = Away Team Hit Woodwork
# HC = Home Team Corners
# AC = Away Team Corners
# HF = Home Team Fouls Committed
# AF = Away Team Fouls Committed
# HFKC = Home Team Free Kicks Conceded
# AFKC = Away Team Free Kicks Conceded
# HO = Home Team Offsides
# AO = Away Team Offsides
# HY = Home Team Yellow Cards
# AY = Away Team Yellow Cards *HR = Home Team Red Cards AR = Away Team Red Cards HBP = Home Team Bookings Points (10 = yellow, 25 = red) ABP = Away Team Bookings Points (10 = yellow, 25 = red)

In [83]:
print(df.columns.values)

['Div' 'Date' 'Time' 'HomeTeam' 'AwayTeam' 'FTHG' 'FTAG' 'FTR' 'HTHG'
 'HTAG' 'HTR' 'Referee' 'HS' 'AS' 'HST' 'AST' 'HF' 'AF' 'HC' 'AC' 'HY'
 'AY' 'HR' 'AR']


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 24 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Div       380 non-null    object
 1   Date      380 non-null    object
 2   Time      380 non-null    object
 3   HomeTeam  380 non-null    object
 4   AwayTeam  380 non-null    object
 5   FTHG      380 non-null    int64 
 6   FTAG      380 non-null    int64 
 7   FTR       380 non-null    object
 8   HTHG      380 non-null    int64 
 9   HTAG      380 non-null    int64 
 10  HTR       380 non-null    object
 11  Referee   380 non-null    object
 12  HS        380 non-null    int64 
 13  AS        380 non-null    int64 
 14  HST       380 non-null    int64 
 15  AST       380 non-null    int64 
 16  HF        380 non-null    int64 
 17  AF        380 non-null    int64 
 18  HC        380 non-null    int64 
 19  AC        380 non-null    int64 
 20  HY        380 non-null    int64 
 21  AY        380 no

In [85]:
df.describe()

,FTHG,FTAG,HTHG,HTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
count,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000
mean,1.634211,1.218421,0.757895,0.563158,13.952632,11.310526,4.907895,3.894737,10.597368,10.931579,5.636842,4.471053,1.671053,1.915789,0.047368,0.026316
std,1.419944,1.183518,0.918480,0.746998,5.604170,4.941173,2.495260,2.230627,3.288020,3.463424,3.073370,2.817530,1.249677,1.358357,0.224768,0.160284
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,10.000000,8.000000,3.000000,2.000000,8.000000,8.000000,3.000000,2.000000,1.000000,1.000000,0.000000,0.000000
50%,1.000000,1.000000,1.000000,0.000000,14.000000,11.000000,5.000000,4.000000,10.000000,11.000000,5.000000,4.000000,2.000000,2.000000,0.000000,0.000000
75%,2.000000,2.000000,1.000000,1.000000,17.000000,15.000000,7.000000,5.000000,13.000000,13.000000,8.000000,6.000000,3.000000,3.000000,0.000000,0.000000
max,9.000000,6.000000,5.000000,3.000000,33.000000,30.000000,15.000000,11.000000,23.000000,24.000000,17.000000,19.000000,6.000000,7.000000,2.000000,1.000000


In [86]:
n_matches = df.shape[0]
n_features = df.shape[1] - 1 
n_homewins = len(df[df.FTR == 'H'])

win_rate = (float(n_homewins)/(n_matches))*100 
print(win_rate)

48.421052631578945


In [87]:
# from pandas.plotting import scatter_matrix
# scatter_matrix(df)

In [88]:
df.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,E0,05/08/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,10,10,2,2,16,11,3,5,1,2,0,0
1,E0,06/08/2022,12:30,Fulham,Liverpool,2,2,D,1,0,H,A Madley,9,11,3,4,7,9,4,4,2,0,0,0
2,E0,06/08/2022,15:00,Bournemouth,Aston Villa,2,0,H,1,0,H,P Bankes,7,15,3,2,18,16,5,5,3,3,0,0
3,E0,06/08/2022,15:00,Leeds,Wolves,2,1,H,1,1,D,R Jones,12,15,4,6,13,9,6,4,2,0,0,0
4,E0,06/08/2022,15:00,Newcastle,Nott'm Forest,2,0,H,0,0,D,S Hooper,23,5,10,0,9,14,11,1,0,3,0,0


In [89]:
df.shape

(380, 24)

In [90]:
df['HomeTeam'].value_counts()

HomeTeam
Crystal Palace    19
Fulham            19
Bournemouth       19
Leeds             19
Newcastle         19
Tottenham         19
Everton           19
Leicester         19
Man United        19
West Ham          19
Aston Villa       19
Arsenal           19
Brighton          19
Man City          19
Southampton       19
Wolves            19
Brentford         19
Nott'm Forest     19
Chelsea           19
Liverpool         19
Name: count, dtype: int64

In [94]:
df['AwayTeam'].value_counts()

AwayTeam
Arsenal           19
Liverpool         19
Aston Villa       19
Wolves            19
Nott'm Forest     19
Southampton       19
Chelsea           19
Brentford         19
Brighton          19
Man City          19
Everton           19
Leicester         19
Newcastle         19
Bournemouth       19
Leeds             19
Fulham            19
Man United        19
West Ham          19
Tottenham         19
Crystal Palace    19
Name: count, dtype: int64

# Datetime

In [103]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

In [108]:
df['Hour'] = df['Time'].apply(lambda x: int(x.split(':')[0]))

In [112]:
df["Day_code"] = df['Date'].dt.dayofweek

In [114]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['homecode'] = le.fit_transform(df['HomeTeam'])
df['awaycode'] = le.transform(df['AwayTeam'])
df.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Hour,Day_code,homecode,awaycode
0,E0,2022-08-05,20:00,Crystal Palace,Arsenal,0,2,A,0,1,A,A Taylor,10,10,2,2,16,11,3,5,1,2,0,0,20,4,6,0
1,E0,2022-08-06,12:30,Fulham,Liverpool,2,2,D,1,0,H,A Madley,9,11,3,4,7,9,4,4,2,0,0,0,12,5,8,11
2,E0,2022-08-06,15:00,Bournemouth,Aston Villa,2,0,H,1,0,H,P Bankes,7,15,3,2,18,16,5,5,3,3,0,0,15,5,2,1
3,E0,2022-08-06,15:00,Leeds,Wolves,2,1,H,1,1,D,R Jones,12,15,4,6,13,9,6,4,2,0,0,0,15,5,9,19
4,E0,2022-08-06,15:00,Newcastle,Nott'm Forest,2,0,H,0,0,D,S Hooper,23,5,10,0,9,14,11,1,0,3,0,0,15,5,14,15


In [121]:
le.inverse_transform(list(range(0,20)))

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton',
       'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leeds',
       'Leicester', 'Liverpool', 'Man City', 'Man United', 'Newcastle',
       "Nott'm Forest", 'Southampton', 'Tottenham', 'West Ham', 'Wolves'],
      dtype=object)

In [165]:
# Linear model cannot makeout the teams number is not ordinal 
from sklearn.ensemble import RandomForestClassifier 
rf = RandomForestClassifier(n_estimators=50, min_samples_leaf=4, random_state=1)

In [166]:
train = df[:-50]
test = df[-50:]

In [167]:
features = ["Hour", "Day_code", "homecode", "awaycode"]

In [168]:
rf.fit(train[features], train['FTR'])

RandomForestClassifier(min_samples_leaf=4, n_estimators=50, random_state=1)

In [169]:
rf.predict(test[features])

array(['H', 'H', 'H', 'H', 'H', 'H', 'D', 'H', 'H', 'H', 'A', 'H', 'H',
       'A', 'H', 'H', 'H', 'A', 'H', 'H', 'H', 'H', 'A', 'H', 'A', 'H',
       'A', 'H', 'A', 'H', 'H', 'H', 'H', 'A', 'A', 'H', 'H', 'H', 'D',
       'H', 'H', 'H', 'H', 'D', 'H', 'H', 'H', 'H', 'H', 'A'],
      dtype=object)

In [ ]:
test['FTR']

In [171]:
from sklearn.metrics import classification_report
print(classification_report(test['FTR'], rf.predict(test[features])))

              precision    recall  f1-score   support

           A       0.40      0.40      0.40        10
           D       0.67      0.18      0.29        11
           H       0.65      0.83      0.73        29

    accuracy                           0.60        50
   macro avg       0.57      0.47      0.47        50
weighted avg       0.60      0.60      0.56        50



In [172]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test['FTR'], rf.predict(test[features]))

array([[ 4,  0,  6],
       [ 2,  2,  7],
       [ 4,  1, 24]])

# Feature engineering 

In [173]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("Date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group 

In [177]:
cols = ['FTHG','FTAG','HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY','AY', 'HR', 'AR']
new_cols_h = [f"{c}_rolling_ht" for c in cols]
new_cols_a = [f"{c}_rolling_at" for c in cols]

In [188]:
matches_rolling_h = df.groupby("HomeTeam").apply(lambda x: rolling_averages(x, cols, new_cols_h))
matches_rolling_h = matches_rolling_h.droplevel('HomeTeam')

C:\Users\shouv\AppData\Local\Temp\ipykernel_40152\553221324.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling_h = df.groupby("HomeTeam").apply(lambda x: rolling_averages(x, cols, new_cols_h))


In [189]:
matches_rolling_h = matches_rolling_h.groupby("AwayTeam").apply(lambda x: rolling_averages(x, cols, new_cols_a))
matches_rolling_h = matches_rolling_h.droplevel('AwayTeam')

C:\Users\shouv\AppData\Local\Temp\ipykernel_40152\3882690442.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling_h = matches_rolling_h.groupby("AwayTeam").apply(lambda x: rolling_averages(x, cols, new_cols_a))


In [193]:
matches_rolling_h.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,Hour,Day_code,homecode,awaycode,FTHG_rolling_ht,FTAG_rolling_ht,HS_rolling_ht,AS_rolling_ht,HST_rolling_ht,AST_rolling_ht,HF_rolling_ht,AF_rolling_ht,HC_rolling_ht,AC_rolling_ht,HY_rolling_ht,AY_rolling_ht,HR_rolling_ht,AR_rolling_ht,FTHG_rolling_at,FTAG_rolling_at,HS_rolling_at,AS_rolling_at,HST_rolling_at,AST_rolling_at,HF_rolling_at,AF_rolling_at,HC_rolling_at,AC_rolling_at,HY_rolling_at,AY_rolling_at,HR_rolling_at,AR_rolling_at
131,E0,2022-11-06,12:00,Chelsea,Arsenal,0,1,A,0,0,D,M Oliver,5,14,1,2,20,13,4,6,5,2,0,0,12,6,5,0,2.000000,0.666667,11.333333,9.000000,4.000000,3.666667,7.333333,9.666667,6.666667,3.000000,2.333333,2.000000,0.000000,0.000000,0.333333,1.666667,10.333333,11.333333,3.000000,4.666667,10.333333,9.666667,6.000000,3.333333,0.666667,1.666667,0.0,0.0
143,E0,2022-11-12,19:45,Wolves,Arsenal,0,2,A,0,0,D,S Attwell,11,14,2,4,5,6,6,4,3,2,0,0,19,5,19,0,1.000000,2.333333,12.666667,11.333333,4.000000,4.666667,14.000000,12.333333,5.666667,5.333333,2.000000,1.000000,0.333333,0.000000,0.333333,1.000000,10.333333,11.666667,2.666667,3.000000,13.666667,10.666667,6.333333,4.333333,2.333333,1.666667,0.0,0.0
163,E0,2022-12-31,17:30,Brighton,Arsenal,2,4,A,0,2,A,A Taylor,9,14,5,7,7,11,7,4,3,3,0,0,17,5,4,0,1.666667,1.000000,15.000000,8.666667,6.000000,3.000000,13.333333,10.333333,9.333333,4.000000,2.000000,3.333333,0.000000,0.000000,0.333333,1.333333,8.666667,13.333333,2.000000,3.000000,12.000000,9.000000,6.333333,4.666667,3.333333,1.666667,0.0,0.0
186,E0,2023-01-15,16:30,Tottenham,Arsenal,0,2,A,0,2,A,C Pawson,17,14,7,5,16,15,4,3,4,2,0,0,16,6,17,0,1.666667,2.333333,11.333333,12.666667,5.000000,5.000000,11.000000,8.333333,6.333333,2.666667,1.666667,1.333333,0.000000,0.333333,0.666667,2.333333,8.333333,14.000000,2.666667,4.333333,10.666667,10.000000,5.666667,4.666667,3.666667,2.333333,0.0,0.0
200,E0,2023-02-04,12:30,Everton,Arsenal,1,0,H,0,0,D,D Coote,12,15,4,3,13,8,7,5,4,1,0,0,12,5,7,0,1.000000,2.666667,11.333333,13.000000,4.666667,5.666667,8.000000,17.000000,4.333333,2.666667,1.333333,3.333333,0.000000,0.000000,0.666667,2.666667,12.333333,14.000000,4.666667,5.333333,9.333333,10.666667,5.666667,3.666667,3.333333,2.333333,0.0,0.0


In [232]:
matches = matches_rolling_h.sort_values(['Date','Time'])

# Model Building

In [243]:
features = ["Hour", "Day_code", "homecode", "awaycode"]  
feat_set = features + new_cols_h + new_cols_a

In [250]:
# Linear model cannot makeout the teams number is not ordinal 
from sklearn.ensemble import RandomForestClassifier 
rf = RandomForestClassifier(n_estimators=50, min_samples_leaf=4, random_state=1)

In [251]:
train = matches[:-80]
test = matches[-80:]

In [252]:
rf.fit(train[feat_set], train['FTR'])

RandomForestClassifier(min_samples_leaf=4, n_estimators=50, random_state=1)

In [253]:
y_pred = list(rf.predict(test[feat_set]))

In [254]:
from sklearn.metrics import classification_report
print(classification_report(test['FTR'], rf.predict(test[feat_set])))

              precision    recall  f1-score   support

           A       0.35      0.30      0.32        20
           D       0.00      0.00      0.00        17
           H       0.61      0.88      0.72        43

    accuracy                           0.55        80
   macro avg       0.32      0.39      0.35        80
weighted avg       0.42      0.55      0.47        80



In [255]:
from sklearn.metrics import confusion_matrix
confusion_matrix(list(test['FTR']), y_pred)

array([[ 6,  0, 14],
       [ 7,  0, 10],
       [ 4,  1, 38]])